In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Using device: cpu
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Data Processing and Feature Engineering

In [4]:
# Combine for consistent feature engineering
full = pd.concat([train_data, test_data], sort=False).reset_index(drop=True)

# Fill missing values
full["Age"] = full["Age"].fillna(full["Age"].median())
full["Fare"] = full["Fare"].fillna(full["Fare"].median())
full["Embarked"] = full["Embarked"].fillna(full["Embarked"].mode()[0])

# Family size & being alone (same as your model2)
full["FamilySize"] = full["SibSp"] + full["Parch"] + 1
full["IsAlone"] = (full["FamilySize"] == 1).astype(int)

# Feature list
features = [
    "Pclass",
    "Sex",
    "Age",
    "Fare",
    "Embarked",
    "SibSp",
    "Parch",
    "FamilySize",
    "IsAlone",
]

# Split back into train / test
full_train = full.iloc[:len(train_data)]
full_test  = full.iloc[len(train_data):]

X = full_train[features]
X_test = full_test[features]
y = train_data["Survived"]

# One-hot encode categorical variables
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test)

# Align columns between train and test
X, X_test = X.align(X_test, join="left", axis=1)
X_test = X_test.fillna(0)

print("Train features shape:", X.shape)
print("Test features shape :", X_test.shape)


Train features shape: (891, 12)
Test features shape : (418, 12)


## Scale features

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)


## Split for Validation

In [6]:
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train size:", X_train.shape)
print("Val size  :", X_val.shape)


Train size: (712, 12)
Val size  : (179, 12)


## PyTorch Tensors & DataLoaders

In [7]:
# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1)

# Datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset   = TensorDataset(X_val_tensor, y_val_tensor)

# DataLoaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


## Define the Neural Network

In [8]:
class TitanicNet(nn.Module):
    def __init__(self, input_dim):
        super(TitanicNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3),

            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.3),

            nn.Linear(32, 16),
            nn.ReLU(),

            nn.Linear(16, 1)  # output logits for BCEWithLogitsLoss
        )

    def forward(self, x):
        return self.net(x)

input_dim = X_train.shape[1]
model = TitanicNet(input_dim).to(device)
print(model)


TitanicNet(
  (net): Sequential(
    (0): Linear(in_features=12, out_features=64, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=32, out_features=16, bias=True)
    (9): ReLU()
    (10): Linear(in_features=16, out_features=1, bias=True)
  )
)


## Set Up Loss Function, Optimizer, and Training Loop with Early Stopping

In [9]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 100
patience = 10  # early stopping patience

best_val_loss = np.inf
epochs_no_improve = 0
best_state_dict = None

for epoch in range(num_epochs):
    # ========= Train =========
    model.train()
    train_losses = []

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    avg_train_loss = np.mean(train_losses)

    # ========= Validate =========
    model.eval()
    val_losses = []
    y_val_true = []
    y_val_pred = []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            logits = model(X_batch)
            loss = criterion(logits, y_batch)
            val_losses.append(loss.item())

            probs = torch.sigmoid(logits)
            preds = (probs > 0.5).float()

            y_val_true.extend(y_batch.cpu().numpy())
            y_val_pred.extend(preds.cpu().numpy())

    avg_val_loss = np.mean(val_losses)
    val_acc = accuracy_score(y_val_true, y_val_pred)

    print(
        f"Epoch {epoch+1:03d} | "
        f"Train Loss: {avg_train_loss:.4f} | "
        f"Val Loss: {avg_val_loss:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )

    # Early stopping check
    if avg_val_loss < best_val_loss - 1e-4:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        best_state_dict = model.state_dict()
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

# Load best weights
if best_state_dict is not None:
    model.load_state_dict(best_state_dict)


Epoch 001 | Train Loss: 0.6534 | Val Loss: 0.6358 | Val Acc: 0.7709
Epoch 002 | Train Loss: 0.5789 | Val Loss: 0.5409 | Val Acc: 0.7765
Epoch 003 | Train Loss: 0.5061 | Val Loss: 0.4877 | Val Acc: 0.8101
Epoch 004 | Train Loss: 0.4724 | Val Loss: 0.4645 | Val Acc: 0.7989
Epoch 005 | Train Loss: 0.4474 | Val Loss: 0.4546 | Val Acc: 0.8212
Epoch 006 | Train Loss: 0.4354 | Val Loss: 0.4505 | Val Acc: 0.8101
Epoch 007 | Train Loss: 0.4372 | Val Loss: 0.4422 | Val Acc: 0.8101
Epoch 008 | Train Loss: 0.4504 | Val Loss: 0.4371 | Val Acc: 0.8156
Epoch 009 | Train Loss: 0.4295 | Val Loss: 0.4392 | Val Acc: 0.8268
Epoch 010 | Train Loss: 0.4434 | Val Loss: 0.4441 | Val Acc: 0.8212
Epoch 011 | Train Loss: 0.4103 | Val Loss: 0.4425 | Val Acc: 0.8212
Epoch 012 | Train Loss: 0.4152 | Val Loss: 0.4446 | Val Acc: 0.8268
Epoch 013 | Train Loss: 0.4354 | Val Loss: 0.4477 | Val Acc: 0.8324
Epoch 014 | Train Loss: 0.4165 | Val Loss: 0.4401 | Val Acc: 0.8324
Epoch 015 | Train Loss: 0.4138 | Val Loss: 0.443

## Retrain on Full Training Data

In [10]:
# Full training data tensors
X_full_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_full_tensor = torch.tensor(y.values, dtype=torch.float32).unsqueeze(1)

full_dataset = TensorDataset(X_full_tensor, y_full_tensor)
full_loader  = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)

final_model = TitanicNet(input_dim).to(device)
final_criterion = nn.BCEWithLogitsLoss()
final_optimizer = torch.optim.Adam(final_model.parameters(), lr=1e-3)

final_epochs = 40  # e.g. fixed number

for epoch in range(final_epochs):
    final_model.train()
    train_losses = []
    for X_batch, y_batch in full_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        final_optimizer.zero_grad()
        logits = final_model(X_batch)
        loss = final_criterion(logits, y_batch)
        loss.backward()
        final_optimizer.step()

        train_losses.append(loss.item())

    print(f"Final Train Epoch {epoch+1:03d} | Loss: {np.mean(train_losses):.4f}")


Final Train Epoch 001 | Loss: 0.6406
Final Train Epoch 002 | Loss: 0.5517
Final Train Epoch 003 | Loss: 0.5033
Final Train Epoch 004 | Loss: 0.4836
Final Train Epoch 005 | Loss: 0.4503
Final Train Epoch 006 | Loss: 0.4522
Final Train Epoch 007 | Loss: 0.4366
Final Train Epoch 008 | Loss: 0.4359
Final Train Epoch 009 | Loss: 0.4251
Final Train Epoch 010 | Loss: 0.4356
Final Train Epoch 011 | Loss: 0.4293
Final Train Epoch 012 | Loss: 0.4135
Final Train Epoch 013 | Loss: 0.4187
Final Train Epoch 014 | Loss: 0.3946
Final Train Epoch 015 | Loss: 0.4096
Final Train Epoch 016 | Loss: 0.3910
Final Train Epoch 017 | Loss: 0.4166
Final Train Epoch 018 | Loss: 0.4203
Final Train Epoch 019 | Loss: 0.4188
Final Train Epoch 020 | Loss: 0.4098
Final Train Epoch 021 | Loss: 0.4125
Final Train Epoch 022 | Loss: 0.4196
Final Train Epoch 023 | Loss: 0.4079
Final Train Epoch 024 | Loss: 0.4146
Final Train Epoch 025 | Loss: 0.4106
Final Train Epoch 026 | Loss: 0.4023
Final Train Epoch 027 | Loss: 0.3967
F

## Make Prediction and Create Submission

In [11]:
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

final_model.eval()
with torch.no_grad():
    logits_test = final_model(X_test_tensor)
    probs_test = torch.sigmoid(logits_test)
    preds_test = (probs_test > 0.5).float().cpu().numpy().ravel()

submission = pd.DataFrame({
    "PassengerId": test_data["PassengerId"],
    "Survived": preds_test.astype(int)
})

submission.to_csv("submission_pytorch_dl.csv", index=False)
print("Submission file 'submission_pytorch_dl.csv' was successfully saved!")


Submission file 'submission_pytorch_dl.csv' was successfully saved!
